In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

In [3]:
def create_assistant(name,instruction,model):
    assistant=client.beta.assistants.create(
    name=name,
    instructions=instruction,
    model=model
    )
    return assistant

In [9]:
def create_thread():
    thread = client.beta.threads.create()
    return thread

In [13]:
def create_message(thread_id,role,content):
    message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role=role,
    content=content
)

In [8]:
def run_assistant(thread,assistant):
    run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
    )

    messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

    message_content = messages[0].content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
        message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
        if file_citation := getattr(annotation, "file_citation", None):
            cited_file = client.files.retrieve(file_citation.file_id)
            citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

In [14]:
#Initialization

assistant=create_assistant("Intelligent Document Processing Assistant","You are intellignet document processing assistant, fetch the data from the documents and analyse it.","gpt-3.5-turbo")
thread=create_thread()
create_message(thread.id,"user","""Use the schema to extract the data from the context and analyse it.{
{"$schema": "http://json-schema.org/draft-07/schema#",
"title": "Generated schema for Root",
"type": "array",
"items": {
"type": "object",
"properties": {
    "name": {
        "type": "string",
        "description": "Name of the lender"
    },
    "term_loan_commitment": {
        "type": "string",
        "description": "Individual commitment of the lender to the project"
    },
    "total_commitment": {
        "type": "string",
        "description": "Total commitment of the lender to the project"
    },
    "percentage": {
        "type": "string",
        "description": "Percentage of the total commitment of the lender to the project"
    },
    "currency": {
        "type": "string",
        "description": "Currency of the commitment amount"
    }
    "Commitment bank": {
        "type": "string",
        "description": "Name of the project for which the commitment is made"
    }
},
"required": [
    "name",
    "term_loan_commitment",
    "total_commitment",
    "percentage",
    "currency"
]
}
}""")

In [17]:

create_message(thread.id,"user","""
context: Use the above schema to extract the data from the below text and analyse it.and give a json output.
Citibank ,Inc
Loan Syndicate distribution Document

Bank Name Amount($)
Individual Commitment
Bank of Baroda 2,000,000

Bank of America 5,000,000

Wells Fargo 8,000,000

Capital One 5,000,000

Total Commitment - $20,000,000

Loan allocated for Citibank.

""")
    

In [18]:
run_assistant(thread,assistant)

Based on the provided text and schema, the extracted data can be structured and analysed as follows:

```json
[
    {
        "name": "Bank of Baroda",
        "term_loan_commitment": "2,000,000",
        "total_commitment": "20,000,000",
        "percentage": "10%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Bank of America",
        "term_loan_commitment": "5,000,000",
        "total_commitment": "20,000,000",
        "percentage": "25%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Wells Fargo",
        "term_loan_commitment": "8,000,000",
        "total_commitment": "20,000,000",
        "percentage": "40%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Capital One",
        "term_loan_commitment": "5,000,000",
        "total_commitment": "20,000,000",
        "percentage": "25%",
        "currency": "USD",
        "Commitment bank":

In [19]:
create_message(thread.id,"user","""
context: Use the above schema to extract the data from the below text and analyse it.and give a json output.
3/29/24, 12:09 PM AMENDED AND RESTATED TERM LOAN AGREEMENT Dated as of September 28, 201... | Law Insider

Schedule I

Commitments

Lender Commitment

JPMORGAN CHASE BANK, N.A $ 70,000,000
SUNTRUST BANK $ 70,000,000
U.S. BANK NATIONAL ASSOCIATION $ 50,000,000
XXXXX FARGO BANK, N.A. $ 60,000,000
PNC BANK, NATIONAL ASSOCIATION $ 35,000,000
BRANCH BANKING AND TRUST COMPANY $ 15,000,000

TOTAL $ 300,000,000

https://www.lawinsider.com/contracts/fRJSRgoKkjk 1/2
3/29/24, 12:09 PM AMENDED AND RESTATED TERM LOAN AGREEMENT Dated as of September 28, 201... | Law Insider

Schedule 1.1(a)

List of Loan Parties

Borrower: Piedmont Operating Partnership, LP, a Delaware limited partnership

Parent: Piedmont Office Realty Trust, Inc., a Maryland corporation

https://www.lawinsider.com/contracts/fRJSRgoKkjk 2/2

""")

In [20]:
run_assistant(thread,assistant)

Based on the provided text and schema, the extracted data can be structured and analysed as follows:

```json
[
    {
        "name": "JPMORGAN CHASE BANK, N.A",
        "term_loan_commitment": "70,000,000",
        "total_commitment": "300,000,000",
        "percentage": "23.33%",
        "currency": "USD",
        "Commitment bank": "Piedmont Office Realty Trust, Inc."
    },
    {
        "name": "SUNTRUST BANK",
        "term_loan_commitment": "70,000,000",
        "total_commitment": "300,000,000",
        "percentage": "23.33%",
        "currency": "USD",
        "Commitment bank": "Piedmont Office Realty Trust, Inc."
    },
    {
        "name": "U.S. BANK NATIONAL ASSOCIATION",
        "term_loan_commitment": "50,000,000",
        "total_commitment": "300,000,000",
        "percentage": "16.67%",
        "currency": "USD",
        "Commitment bank": "Piedmont Office Realty Trust, Inc."
    },
    {
        "name": "XXXXX FARGO BANK, N.A.",
        "term_loan_commitment": "60,00

In [21]:
create_message(thread.id,"user","compare last two outputs.")

In [23]:
run_assistant(thread,assistant)

Comparing the last two outputs:

1. Output 1 (Provided in the first response):
```json
[
    {
        "name": "Bank of Baroda",
        "term_loan_commitment": "2,000,000",
        "total_commitment": "20,000,000",
        "percentage": "10%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Bank of America",
        "term_loan_commitment": "5,000,000",
        "total_commitment": "20,000,000",
        "percentage": "25%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Wells Fargo",
        "term_loan_commitment": "8,000,000",
        "total_commitment": "20,000,000",
        "percentage": "40%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    },
    {
        "name": "Capital One",
        "term_loan_commitment": "5,000,000",
        "total_commitment": "20,000,000",
        "percentage": "25%",
        "currency": "USD",
        "Commitment bank": "Citibank"
    }
]
```